# dunder 魔法方法

>- <font color='red'>dunder</font>是「double under」（雙底線）的縮寫，指的是 Python 中的特殊方法，這類方法的名稱以兩條底線（under line）開頭和結束，也稱為「魔法方法」。

>- Python 定義了許多這類型的方法，可用來更改或拓展（extend）物件的內建行為，也就是說魔法方法可以讓使用者定義 Python 的內建行為，如「加減乘除」。

>- 舉例說，可以在自定義類別中實作名稱為「\_\_add\_\_」的魔法方法，當使用「+」運算子對類的實體進行運算，Python 就會自動調用這個「\_\_add\_\_」方法。

>- 在 \_\_repr\_\_ 部分，官方特別說明了其調用 eval() 進行物件復刻的時候可能發生的問題是可以理解的，原文是「如果可能，這（指 \_\_repr\_\_）應該看起來就像一個 Python 表達式，能用來重新創建該（或者一個相似的）物件。」，而「如果可能」就是告訴開發者盡力而為，但例外也是可以接受的。

1. 對象初始化 \_\_init\_\_(self, ...)

>- 用於初始化一個新建立的物件，當物件被創建時會立即調用 \_\_init\_\_。

In [16]:
# 自定義一個類
class Example:
    # 定義一個初始化方法
    def __init__(self, value):
        # 初始化屬性
        self.value = value

2. 相加 \_\_add\_\_

>- 明白初始化時會調用 \_\_init\_\_，那就不難明白當類實作了 \_\_add\_\_，那類的兩個實體以「+」運算子相加時，就會自動調用 \_\_add\_\_ 方法了。

>- 注意這個結果最終顯示的是一個十六進位的記憶體位置，這是因為這個類沒有實作 \_\_str\_\_ 或是 \_\_repr\_\_，後面部分會接續說明這兩個魔法方法。

>- 透過對結果的以 id 查看發現，沒有實作 \_\_str\_\_ 或是 \_\_repr\_\_ 時所輸出的就是記憶體位置的十六進位表示結果。

In [17]:
class MyNumber:
    def __init__(self, value):
        # 有一個初始化屬性
        self.value = value
    # 定義一個方法，用來實現兩個 MyNumber 物件的加法
    def __add__(self, other):
        # 透過 isinstance() 判斷 other 是否為 MyNumber 類型
        if isinstance(other, MyNumber):
            # 返回兩個物件的 value 屬性之和
            # 返回的是一個新的 MyNumber 物件
            return MyNumber(self.value + other.value)
        else:
            # 表示這個操作未被實現，Python 會嘗試調用其他方法來執行這個操作
            return NotImplemented  

# 創建兩個 MyNumber 物件
n1 = MyNumber(100)
n2 = MyNumber(200)
# 運算 n1 + n2，實際上是調用 n1.__add__(n2)
# 因為沒有實作 __str__ 或是 __repr__，所以會印出物件的記憶體位址
result = n1 + n2
print("--------------------")
# 會輸出該物件所屬的類別和該物件的記憶體地址。
print(result) 
# 印出 result 的記憶體位址的十六進位結果進行比對，發現一致
print(hex(id(result)))
print("--------------------")

--------------------
0x7f85f0fd3cd0
--------------------


3. 官方說明 \_\_repr\_\_
   
>- 再拓展前一個範例，希望輸出的結果是類的實體 MyNumber(300)。

>- 這次將一個 MyNumber 物件傳遞給 print 函數輸出時，Python 會使用 \_\_repr\_\_ 方法來生成一個字符串並打印出來。

In [19]:
class MyNumber:
    def __init__(self, value):
        self.value = value

    def __add__(self, other):
        if isinstance(other, MyNumber):
            return MyNumber(self.value + other.value)
        else:
            return NotImplemented
    # 定義 __repr__ 方法，用來返回對象的字符串表示
    def __repr__(self):
        # 返回一個字符串，這個字符串可以用來創建一個和當前物件相同的新物件
        return f"__repr__：MyNumber({self.value})"  

# 創建兩個 MyNumber 物件
n1 = MyNumber(100)
n2 = MyNumber(200)
# 調用 n1 + n2，實際上是調用 n1.__add__(n2)
# 輸出 MyNumber(300)
result = n1 + n2
print(result) 

__repr__：MyNumber(300)


4. 字串表達形式 \_\_str\_\_(self) 、 \_\_repr\_\_(self)

>- 將一個物件傳遞給 print() 時，Python 會嘗試將該物件轉換為一個字符串並打印。默認情況下，Python 會打印出一個通用的物件表示，包括該物件所屬的類別和該物件的記憶體地址。

>- 在內建類或是自定義類中假如實作了 \_\_str\_\_ 或 \_\_repr\_\_ 魔法方法，那調用 print() 的時候則會調用他們來傳出一段字串提供輸出。

>- 通常 \_\_str\_\_ 會傳出人類可讀的描述（字串），而 \_\_repr\_\_ 則會傳出機器可讀的描述，並用於建立一個相同內容的新物件可用的字串，所以他們需要傳出的訊息量並不相同，設計時也會有所差異。

>- 系統會優先調用 \_\_str\_\_ ，所以當 \_\_str\_\_ 方法沒有定義，那 Python 將會調用 \_\_repr\_\_ 方法，換句話說假如只定義其中之一，那 \_\_repr\_\_ 會是較好的選擇。

>- 以日期物件為例，在 \_\_str\_\_ 中可能回傳的只有「2023-06-12」，但在 \_\_repr\_\_ 中則會傳回更詳細的「datetime.date(2023, 6, 12)」。

>- 這是因為 \_\_repr\_\_ 可用於調用 \_\_eval\_\_ 來建立一個全新但是內容相同的建物，因為這樣的需求，所以在 \_\_repr\_\_ 中的要求更為嚴謹。

>- 針對 \_\_eval\_\_ 再做些說明，並不是說 \_\_str\_\_ 不能調用 \_\_eval\_\_，而是 \_\_str\_\_ 本應被期待傳出人類可以閱讀的訊息，而 \_\_repr\_\_ 則被期待傳出可供機器閱讀並用來重建、復刻原物件的訊息，在程式語言中它們有各自的功能角色，設計者將它們正確合理的設計也是很重要的工作。

>>示範一下 eval()

>- eval() 會執行傳入字串所代表的 <font color='red'>Python 程式碼</font>，而非所顯示的字串。

>- 補充說明，除了 eval() 以外， exec() 函數也可以用來執行 code object，詳見「code object」一節說明。

In [28]:
x1 = '1+2'
x2 = eval(x1)
print(x1, type(x1)) 
print(x2, type(x2)) 

1+2 <class 'str'>
3 <class 'int'>


>- 期待 \_\_repr\_\_ 可以被 eval()所解析。

>- 補充說明一下，在邏輯上，\_\_repr\_\_ 在定義的時候就該定義一個可供 eval() 所解析的內容，所以按理說並不存在 eval() 無法重建 \_\_repr\_\_ 物件這樣的問題，因為這個所謂的無法重建其實來自於 \_\_repr\_\_ 的定義錯誤。

>- 但在實務上，eval() 是 Python 內建的函數，並非自定義類之內所需實作的，當建立了一些複雜的類且實務上無法使用 eval() 重建，並不能否定這樣的類的存在意義，也為了提供人類與機器可閱讀的描述而實作了 \_\str\_\_ 與 \_\_repr\_\_，導致於 \_\_repr\_\_ 無法調用 eval() 來進行重建是合理的存在，這在官方說明中也都有描述。

In [31]:
# For a list
x = [1, 2, 3]
assert eval(repr(x)) == x

# For a dictionary
x = {'a': 1, 'b': 2}
assert eval(repr(x)) == x

>>進一步說明

>- 假如沒有定義 \_\_repr\_\_ 與 \_\_str\_\_，結果：類名＋記憶體位置

>>> <\_\_main\_\_.MyNumber object at 0x7f85f0e53df0>

>- 假如僅定義 \_\_repr\_\_，結果

>>> \_\_repr\_\_：MyNumber(300)

>- 假如僅定義 \_\_str\_\_，結果

>>> \_\_str\_\_：value = 300

>- 假如定義 \_\_repr\_\_ 與 \_\_str\_\_，結果

>>> \_\_str\_\_：value = 300

In [20]:
# 自定義類
class MyNumber:
    # 定義初始化方法
    def __init__(self, value):
        # 初始化屬性
        self.value = value
    # 定義 __add__ 方法，用來實現兩個 MyNumber 物件的加法
    def __add__(self, other):
        # 判斷 other 是否是 MyNumber 的實體
        if isinstance(other, MyNumber):
            # 返回兩個物件的 value 屬性之和
            return MyNumber(self.value + other.value)
        else:
            # 表示這個操作未被實現，Python 會嘗試調用其他方法來執行這個操作
            return NotImplemented
    # 定義 __repr__ 方法，用來返回對象的字符串表示
    def __repr__(self):
        # 返回一個字符串，這個字符串可以用來創建一個和當前物件相同的新物件
        return f"__repr__：MyNumber({self.value})"  
    # 定義 __str__ 方法，用來返回對象的字符串表示
    def __str__(self):
        # 返回一個字符串，這個字符串用來表示當前物件
        return f"__str__：value = {self.value}"

# 創建兩個 MyNumber 物件
n1 = MyNumber(100)
n2 = MyNumber(200)
# 調用 n1 + n2，實際上是調用 n1.__add__(n2)
# 輸出 MyNumber(300)
print(n1 + n2) 

__str__：value = 300


In [18]:
class Example:
    # 定義一個初始化方法
    def __init__(self, value):
        self.value = value

    # 定義一個字串方法
    def __str__(self):
        # 回傳字串
        return "這是一個範例物件"
    
    def __repr__(self):
        return f"Example({self.value})"
    
#
obj = Example(5)
print(obj)
# 使用 repr() 函數
print(repr(obj))
# 透過「!r」
print(f'{obj!r}') 


這是一個範例物件
Example(5)
Example(5)


2. 物件呼叫 \_\_call\_\_(self, ...)

>- 讓一個類的實體(instance)像函數一樣被呼叫。


In [ ]:
class Example:
    # 定義一個呼叫方法
    def __call__(self, value):
        # 回傳值
        return value * 2

---END---